In [2]:
import pandas as pd
from datetime import datetime
import requests

In [5]:
"""
Met à jour les données sur les crimes en récupérant les dernières entrées depuis l'API.
"""
# Charger le DataFrame existant
crimes_file_path_raw = '/home/ml/Chicago_Crime/data/raw/'

df_crimes = pd.read_csv(crimes_file_path_raw + "Crimes_Chicago.csv", usecols=['Date', 'Primary Type', 'Community Area'], parse_dates=['Date'], low_memory=False)
 # Trouver la date la plus récente dans le fichier CSV
last_date = df_crimes['Date'].max()
start_date = last_date + pd.Timedelta(days=1)  # Commencer le jour suivant la dernière dat
 # Convertir start_date au format attendu par l'API (YYYY-MM-DD)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = datetime.now().strftime('%Y-%m-%d')  # Utiliser la date actuelle comme date de fi
 

In [6]:
print(start_date)

2023-02-07 23:57:00


In [7]:
print(end_date_str)

2024-02-16


In [8]:
dict_rename = {"Date": "date", "Primary Type" : "primary_type", 'Community Area' : "community_area"}

In [9]:
df_crimes.rename(columns=dict_rename, inplace=True)

In [10]:
df_crimes

,date,primary_type,community_area
0,2007-08-25 09:22:18,ASSAULT,1.0
1,2021-05-24 15:06:00,HOMICIDE,19.0
2,2021-06-26 09:24:00,HOMICIDE,13.0
3,2002-01-05 21:24:00,THEFT,NaN
4,2019-04-21 12:30:00,ROBBERY,44.0
...,...,...,...
7739318,2023-01-01 09:00:00,THEFT,77.0
7739319,2023-01-24 12:00:00,CRIMINAL SEXUAL ASSAULT,32.0
7739320,2022-03-01 00:00:00,BATTERY,33.0
7739321,2023-02-05 09:00:00,DECEPTIVE PRACTICE,58.0


In [11]:
df_crimes['community_area'] = df_crimes['community_area'].astype('Int64')

In [12]:
df_crimes

,date,primary_type,community_area
0,2007-08-25 09:22:18,ASSAULT,1
1,2021-05-24 15:06:00,HOMICIDE,19
2,2021-06-26 09:24:00,HOMICIDE,13
3,2002-01-05 21:24:00,THEFT,<NA>
4,2019-04-21 12:30:00,ROBBERY,44
...,...,...,...
7739318,2023-01-01 09:00:00,THEFT,77
7739319,2023-01-24 12:00:00,CRIMINAL SEXUAL ASSAULT,32
7739320,2022-03-01 00:00:00,BATTERY,33
7739321,2023-02-05 09:00:00,DECEPTIVE PRACTICE,58


In [18]:
base_url = "https://data.cityofchicago.org/resource/ijzp-q8t2.json"
params = {
           "$where": f"date >= '{start_date_str}' AND date <= '{end_date_str}'",
            "$select": "date, primary_type, community_area",
           "$limit": 5000  # Limite adaptée pour votre cas d'utilisation
       }

# Effectuer la requête à l'API

response = requests.get(base_url, params=params)
if response.status_code == 200:
    # Charger les nouvelles données dans un DataFrame
    new_data = pd.DataFrame(response.json())
    # Renommer les colonnes selon votre dictionnaire
    #new_data.rename(columns=dicto_rename_crimes, inplace=True)
    # Convertir les colonnes de date au format datetime
    new_data['date'] = pd.to_datetime(new_data['date'])
    # Concaténer avec les anciennes données
    updated_df = pd.concat([df_crimes, new_data], ignore_index=True)
    # Sauvegarder le DataFrame mis à jour dans le fichier CSV
    #updated_df.to_csv(crimes_file_path, index=False)
    print(f"Les données ont été mises à jour avec succès jusqu'au {end_date_str}.")
else:
    print("Erreur lors de la récupération des données depuis l'API.")

Les données ont été mises à jour avec succès jusqu'au 2024-02-16.


In [19]:
updated_df

,date,primary_type,community_area
0,2007-08-25 09:22:18,ASSAULT,1
1,2021-05-24 15:06:00,HOMICIDE,19
2,2021-06-26 09:24:00,HOMICIDE,13
3,2002-01-05 21:24:00,THEFT,<NA>
4,2019-04-21 12:30:00,ROBBERY,44
...,...,...,...
7744318,2023-02-14 11:30:00,CRIMINAL TRESPASS,33
7744319,2023-02-14 11:36:00,INTERFERENCE WITH PUBLIC OFFICER,35
7744320,2023-02-14 11:39:00,DECEPTIVE PRACTICE,23
7744321,2023-02-14 11:40:00,ASSAULT,61


In [20]:
crimes_file_path_processed = '/home/ml/Chicago_Crime/data/processed/'

In [22]:
updated_df.to_csv(crimes_file_path_processed + "Crimes_Chicago.csv", index=False)